In [ ]:
import networkx as nx
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [ ]:
G = nx.read_edgelist('../data/Cit-HepPh.txt', create_using=nx.DiGraph)
G_ig = ig.Graph.from_networkx(G)

# Graph properties

In [ ]:
print('number of nodes', nx.number_of_nodes(G))
print('number of edges', nx.number_of_edges(G))
print('number of selfloops', nx.number_of_selfloops(G))
print('number of weakly connected components', nx.number_weakly_connected_components(G))
print('number of strongly connected components', nx.number_strongly_connected_components(G))
print('average clustering coefficient', nx.average_clustering(G))
print('number of weak triangles', sum(nx.triangles(G.to_undirected()).values())/3)
print('fraction of closed triads', nx.transitivity(G))
print('density of graph', nx.density(G))

# Degree distribution

In [ ]:
# get list of in- and out-degrees
in_degrees = np.array(list(dict(G.in_degree).values()))
out_degrees = np.array(list(dict(G.out_degree).values()))

# get frequency of each degree and normalize to density
d_in, v_in = np.unique(in_degrees, return_counts=True)
v_in = v_in / v_in.sum()
d_out, v_out = np.unique(out_degrees, return_counts=True)
v_out = v_out / v_out.sum()

# setup figure for plotting
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,6))
dot_size = 10

# plot in-degree
ax1.scatter(d_in, v_in, s=dot_size)
ax1.set_xlabel('k')
ax1.set_ylabel('p(k)')
ax1.set_title('in-degree distribution')

# plot in-degree log-log
ax3.scatter(d_in, v_in, s=dot_size)
ax3.set_xlabel('k')
ax3.set_ylabel('p(k)')
ax3.set_title('in-degree distribution')
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlim(0.9, np.max(d_in)*2)

# plot out-degree
ax2.scatter(d_out, v_out, s=dot_size)
ax2.set_xlabel('k')
ax2.set_ylabel('p(k)')
ax2.set_title('out-degree distribution')

# plot out-degree log-log
d_out, v_out = np.unique(out_degrees, return_counts=True)
v_out = v_out / v_out.sum()
ax4.scatter(d_out, v_out, s=dot_size)
ax4.set_xlabel('k')
ax4.set_ylabel('p(k)')
ax4.set_title('out-degree distribution')
ax4.set_xscale('log')
ax4.set_yscale('log')
ax4.set_xlim(0.9, np.max(d_out)*2)

plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,3))

# get complementray cumulative sum of degrees
v_in_cum = 1 - np.cumsum(v_in)
v_out_cum = 1 - np.cumsum(v_out)

# plot in-degree CCDF log-log
ax1.plot(d_in, v_in_cum)
ax1.set_title('in-degree CCDF')
ax1.set_xlabel('k')
ax1.set_ylabel('p(k$\geq$x)')
ax1.set_xscale('log')
ax1.set_yscale('log')

# plot out-degree CCDF log-log
ax2.plot(d_out, v_out_cum)
ax2.set_title('out-degree CCDF')
ax2.set_xlabel('k')
ax2.set_ylabel('p(k$\geq$x)')
ax2.set_xscale('log')
ax2.set_yscale('log')

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(in_degrees, out_degrees, s=dot_size)
ax.set_title('in- and out-degree relationship')
ax.set_xlabel('in-degree')
ax.set_ylabel('out-degree')

In [ ]:
# degree assortativity

# Relevant distributions
Do these for largest component

In [ ]:
# clustering coefficient
clustering = nx.clustering(G.to_undirected())

fig, ax = plt.subplots()
sns.histplot(list(clustering.values()), stat='probability', ax=ax)
ax.set_title('Clustering Coefficient Distribution')
ax.set_xlabel('Clustering Coefficient')
ax.set_ylabel('Frequency')

In [ ]:
# betweenness centrality
betweenness = G_ig.components().giant().betweenness()
betweenness = np.array(betweenness) / (G_ig.vcount() * (G_ig.vcount() - 1))

fig, ax = plt.subplots()
sns.histplot(betweenness, stat='probability', ax=ax, bins=10)
ax.set_title('Betweeness Centrality Distribution')
ax.set_xlabel('Betweeness Centrality')
ax.set_ylabel('Frequency')
ax.set_yscale('log')

print('average betweenness centrality', np.mean(betweenness))

In [ ]:
# closeness centrality
closeness = G_ig.closeness()

fig, ax = plt.subplots()
sns.histplot(closeness, stat='probability', ax=ax)
ax.set_title('Closeness Centrality Distribution')
ax.set_xlabel('Closeness Centrality')
ax.set_ylabel('Frequency')

print('average closeness centrality', np.mean(closeness))

In [ ]:
# pagerank
pagerank = G_ig.pagerank()

fig, ax = plt.subplots()
sns.histplot(pagerank, stat='probability', ax=ax)
ax.set_title('Pagerank Distribution')
ax.set_xlabel('Pagerank')
ax.set_ylabel('Frequency')

print('average pagerank', np.mean(pagerank))

# Cliques

# Community detection
NOTE: Read up on modularity

In [ ]:
# import leidenalg as la
# communities = la.find_partition(G_ig, la.ModularityVertexPartition)
communities = G_ig.community_infomap()

In [ ]:
G_ig.vs['community'] = communities.membership

# Graph summarization

In [ ]:
# do graph aggregation based on communities

# Network visualization

In [ ]:
layout = G_ig.layout("fr")
ig.plot(G_ig, layout=layout, vertex_size=1, vertex_label=None, edge_arrow_size=0.1, edge_width=0.1)

# Connected component analysis

In [ ]:
wcc_components = np.array(list(nx.weakly_connected_components(G)))
wcc_component_sizes = np.array([len(c) for c in wcc_components])
wcc_idx = np.argsort(wcc_component_sizes)[::-1]
wcc_components = wcc_components[wcc_idx]
wcc_component_sizes = wcc_component_sizes[wcc_idx]

scc_components = np.array(list(nx.strongly_connected_components(G)))
scc_component_sizes = np.array([len(c) for c in scc_components])
scc_idx = np.argsort(scc_component_sizes)[::-1]
scc_components = scc_components[scc_idx]
scc_component_sizes = scc_component_sizes[scc_idx]

In [ ]:
print('largest weakly connected component size', wcc_component_sizes[0])
print('second largest weakly connected component size', wcc_component_sizes[1])
print('largest strongly connected component size', scc_component_sizes[0])
print('second largest strongly connected component size', scc_component_sizes[1])

G_wcc = G.subgraph(wcc_components[0])
G_scc = G.subgraph(scc_components[0])